In [8]:
import requests
from secrets_config import api_key_id, api_secret_key
import pandas as pd
from datetime import datetime, timezone

### Extracting the data

Requesting data with the following parameters:

- stock: tesla
- start date: 01/01/2020
- end date: 02/01/2020

In [9]:
# docs: https://alpaca.markets/docs/api-references/market-data-api/stock-pricing-data/historical/

"""
request data with the following parameters: 
- stock: tesla 
- start date: 01/01/2020
- end date: 02/01/2020
"""
stock_ticker = "tsla" # tlsa maps to tesla
base_url = f"https://data.alpaca.markets/v2/stocks/{stock_ticker}/trades"
start_time = datetime(2020, 1, 1, tzinfo=timezone.utc).isoformat()
end_time = datetime(2020, 1, 2, tzinfo=timezone.utc).isoformat()

"""
to authenticate to the api, you will need to use the APCA-API-KEY-ID and APCA-API-SECRET-KEY fields. 
for example:
    headers = {
        "APCA-API-KEY-ID": "<your_api_key_id>",
        "APCA-API-SECRET-KEY": "<your_api_secret_key>"
    }   

and in the request.get() method, you will have to use: 
    requests.get(url=url, params=params, headers=headers)
"""

response_data = []

params = {
    "start": start_time,
    "end": end_time
}

# auth example: https://alpaca.markets/docs/api-references/trading-api/
headers = {
    "APCA-API-KEY-ID": api_key_id,
    "APCA-API-SECRET-KEY": api_secret_key
}
response = requests.get(base_url, params=params, headers=headers)
if response.json().get("trades") is not None: 
    response_data.extend(response.json().get("trades"))

In [10]:
# storing the data into a pandas dataframe

df_quote = pd.json_normalize(data=response_data, meta=["symbol"])

df_quote.head()

,t,x,p,s,c,i,z
0,2020-01-01T00:00:20.4997Z,K,418.93,60,"[@, T, I]",9905,C
1,2020-01-01T00:00:38.5731Z,K,418.70,10,"[@, T, I]",9906,C
2,2020-01-01T00:00:50.8222Z,T,418.90,3,"[@, T, I]",41862,C
3,2020-01-01T00:00:50.8222Z,T,418.58,2,"[@, T, I]",41863,C
4,2020-01-01T00:06:12.9975Z,D,418.75,8,"[@, T, I]",2183581,C


### Renaming the columns to more meaningful names

In [11]:
new_column_names = {
    'i':'id',
    't': 'timestamp',
    'x': 'exchange',
    'p': 'price',
    's':'size'
}

df_quote_renamed = df_quote.rename(columns=new_column_names)

In [12]:
df_quote_renamed.head()

,timestamp,exchange,price,size,c,id,z
0,2020-01-01T00:00:20.4997Z,K,418.93,60,"[@, T, I]",9905,C
1,2020-01-01T00:00:38.5731Z,K,418.70,10,"[@, T, I]",9906,C
2,2020-01-01T00:00:50.8222Z,T,418.90,3,"[@, T, I]",41862,C
3,2020-01-01T00:00:50.8222Z,T,418.58,2,"[@, T, I]",41863,C
4,2020-01-01T00:06:12.9975Z,D,418.75,8,"[@, T, I]",2183581,C


In [13]:
# keep only 'id', 'timestamp', 'exchange', 'price', 'size' columns 
df_quotes_selected = df_quote_renamed[['id', 'timestamp', 'exchange', 'price', 'size']]

In [14]:
df_quotes_selected.head()

,id,timestamp,exchange,price,size
0,9905,2020-01-01T00:00:20.4997Z,K,418.93,60
1,9906,2020-01-01T00:00:38.5731Z,K,418.70,10
2,41862,2020-01-01T00:00:50.8222Z,T,418.90,3
3,41863,2020-01-01T00:00:50.8222Z,T,418.58,2
4,2183581,2020-01-01T00:06:12.9975Z,D,418.75,8


### Joining the exhange codes

In [15]:
df_exchange_codes = pd.read_csv('exchange_codes.csv')

In [16]:
df_exchange_codes.head()

,exchange_code,exchange_name
0,A,NYSE American (AMEX)
1,B,NASDAQ OMX BX
2,C,National Stock Exchange
3,D,FINRA ADF
4,E,Market Independent


In [17]:
df_exchange = pd.merge(left=df_quotes_selected, right=df_exchange_codes, left_on="exchange", right_on="exchange_code").drop(columns=["exchange_code", "exchange"]).rename(columns={"exchange_name": "exchange"})
df_exchange.head()


,id,timestamp,price,size,exchange
0,9905,2020-01-01T00:00:20.4997Z,418.93,60,Cboe EDGX
1,9906,2020-01-01T00:00:38.5731Z,418.70,10,Cboe EDGX
2,9907,2020-01-01T00:09:35.5596Z,418.75,20,Cboe EDGX
3,9908,2020-01-01T00:10:13.0418Z,418.75,115,Cboe EDGX
4,9909,2020-01-01T00:10:28.0684Z,418.75,200,Cboe EDGX
